<div align="center">

# <span style="color: #3498db;">CA2 - Genetic & Game</span>

**<span style="color:rgb(247, 169, 0);">[Student Name]</span> - <span style="color:rgb(143, 95, 195);">[Student Number]</span>**

</div>


# <span style="color: #3498db;">Minmax Algorithm</span>

In [78]:
import random
import numpy as np
from math import inf
import time
import pygame
import matplotlib.pyplot as plt 
from tqdm import tqdm
import pandas as pd

In [79]:
class PentagoGame:
    def __init__(self, ui=False, print=False, depth=2, prune=False):
        self.board = np.zeros((6, 6), dtype=int)
        self.current_player = 1
        self.ui = ui
        self.depth = depth
        self.nodes_visited = 0
        self.game_over = False
        self.result = None
        self.selected_block = None
        self.move_stage = 0  # 0: place piece, 1: select block, 2: rotate
        self.temp_piece = None
        self.print = print
        self.prune = prune

        if ui:
            pygame.font.init()
            self.screen = pygame.display.set_mode((800, 600))
            pygame.display.set_caption("Pygame Board")
            self.font = pygame.font.SysFont("Arial", 20)
            self.font_large = pygame.font.SysFont("Arial", 36)
            self.font_small = pygame.font.SysFont("Arial", 24)
            self.show_buttons = False
            self.buttons = {
                "rotate_cw": pygame.Rect(650, 200, 100, 50),
                "rotate_ccw": pygame.Rect(650, 300, 100, 50),
            }
            self.setup_controls()
            self.draw_board()

    def setup_controls(self):
        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])   # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.draw_text("CLOCKWISE", self.buttons["rotate_cw"].center, self.buttons["rotate_cw"].width)
            self.draw_text("COUNTER-CLOCKWISE", self.buttons["rotate_ccw"].center, self.buttons["rotate_ccw"].width)

    def hide_rotation_buttons(self):
        self.show_buttons = False

    def show_rotation_buttons(self):
        self.show_buttons = True

    def copy_board(self, board):
        return np.copy(board)

    def rotate_block(self, board, block, direction):
        row_start = (block // 2) * 3
        col_start = (block % 2) * 3
        sub = board[row_start : row_start + 3, col_start : col_start + 3]
        rotated = np.rot90(sub, 3 if direction == "cw" else 1)
        board[row_start : row_start + 3, col_start : col_start + 3] = rotated

    def get_possible_moves(self, board, player):
        moves = []
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    for block in range(4):
                        for dir in ["cw", "ccw"]:
                            moves.append((i, j, block, dir))
        return moves

    def apply_move(self, board, move, player):
        new_board = self.copy_board(board)
        row, col, block, direction = move
        if new_board[row][col] != 0:
            return None
        new_board[row][col] = player
        self.rotate_block(new_board, block, direction)
        return new_board

    def check_winner(self, board):
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    continue

                # Horizontal
                if j <= 1 and np.all(board[i, j : j + 5] == board[i][j]):
                    return board[i][j]

                # Vertical
                if i <= 1 and np.all(board[i : i + 5, j] == board[i][j]):
                    return board[i][j]

                # Diagonal
                if (
                    i <= 1
                    and j <= 1
                    and all(board[i + k][j + k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]

                # Anti-diagonal
                if (
                    i <= 1
                    and j >= 4
                    and all(board[i + k][j - k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]
        if np.all(board != 0):
            return 0
        return None

    def evaluate_board(self, board):
        """
        Heuristic function to evaluate the board state.
        Returns higher values for positions favorable to the computer (-1) player.
        """
        winner = self.check_winner(board)
        if winner == -1:  # Computer wins
            return 1000
        elif winner == 1:  # Human wins
            return -1000
        elif winner == 0:  # Draw
            return 0
        
        # Count potential winning lines
        computer_score = 0
        player_score = 0
        
        # Check rows
        for i in range(6):
            for j in range(2):  # Only need to check positions that can start a 5-in-a-row
                line = board[i, j:j+5]
                computer_pieces = np.sum(line == -1)
                player_pieces = np.sum(line == 1)
                
                if player_pieces == 0 and computer_pieces > 0:
                    computer_score += 2 ** computer_pieces
                elif computer_pieces == 0 and player_pieces > 0:
                    player_score += 2 ** player_pieces
        
        # Check columns
        for i in range(2):  # Only need to check positions that can start a 5-in-a-row
            for j in range(6):
                line = board[i:i+5, j]
                computer_pieces = np.sum(line == -1)
                player_pieces = np.sum(line == 1)
                
                if player_pieces == 0 and computer_pieces > 0:
                    computer_score += 2 ** computer_pieces
                elif computer_pieces == 0 and player_pieces > 0:
                    player_score += 2 ** player_pieces
        
        # Check diagonals (top-left to bottom-right)
        for i in range(2):
            for j in range(2):
                line = [board[i+k][j+k] for k in range(5)]
                computer_pieces = sum(1 for piece in line if piece == -1)
                player_pieces = sum(1 for piece in line if piece == 1)
                
                if player_pieces == 0 and computer_pieces > 0:
                    computer_score += 2 ** computer_pieces
                elif computer_pieces == 0 and player_pieces > 0:
                    player_score += 2 ** player_pieces
        
        # Check diagonals (top-right to bottom-left)
        for i in range(2):
            for j in range(4, 6):
                line = [board[i+k][j-k] for k in range(5)]
                computer_pieces = sum(1 for piece in line if piece == -1)
                player_pieces = sum(1 for piece in line if piece == 1)
                
                if player_pieces == 0 and computer_pieces > 0:
                    computer_score += 2 ** computer_pieces
                elif computer_pieces == 0 and player_pieces > 0:
                    player_score += 2 ** player_pieces
        
        # Control of center positions
        center_positions = [(2, 2), (2, 3), (3, 2), (3, 3)]
        for x, y in center_positions:
            if board[x][y] == -1:  # Computer
                computer_score += 3
            elif board[x][y] == 1:  # Player
                player_score += 3
        
        return computer_score - player_score
    
    def minimax_with_pruning(self, board=None, depth=None, alpha=-inf, beta=inf, maximizing_player=True):
        """
        Minimax algorithm with alpha-beta pruning.
        
        Args:
            board: Current board state (if None, use self.board)
            depth: Depth to search to (if None, use self.depth)
            alpha: Alpha value for pruning
            beta: Beta value for pruning
            maximizing_player: True if computer (-1), False if human (1)
            
        Returns:
            Value of the board position
        """
        if board is None:
            board = self.board
                
        if depth is None:
            depth = self.depth
                
        self.nodes_visited += 1

        # Check if game is over or depth limit reached
        winner = self.check_winner(board)
        if winner is not None:
            if winner == -1:  # Computer wins
                return 1000
            elif winner == 1:  # Human wins
                return -1000
            else:  # Draw
                return 0
        
        if depth == 0:
            return self.evaluate_board(board)
        
        # Computer's turn (maximizing)
        if maximizing_player:
            max_eval = -inf
            for move in self.get_possible_moves(board, -1):
                new_board = self.apply_move(board, move, -1)
                if new_board is None:
                    continue
                if self.prune:    
                    eval = self.minimax_with_pruning(new_board, depth - 1, alpha, beta, False)
                else:
                    eval = self.minimax_no_pruning(new_board, depth-1,False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                
                if beta <= alpha:
                    break  # Beta cutoff
                    
            return max_eval
            
        # Human's turn (minimizing)
        else:
            min_eval = inf
            for move in self.get_possible_moves(board, 1):
                new_board = self.apply_move(board, move, 1)
                if new_board is None:
                    continue
                if self.prune:    
                    eval = self.minimax_with_pruning(new_board, depth - 1, alpha, beta, True)
                else:
                    eval = self.minimax_no_pruning(new_board, depth - 1, True)
                    
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                
                if beta <= alpha:
                    break  # Alpha cutoff
                    
            return min_eval
        
    def minimax_no_pruning(self, board, depth, is_maximizer):
        """
        Minimax algorithm without alpha-beta pruning.
        
        Args:
            board: The current game board
            depth: Search depth remaining
            is_maximizer: True if current player is maximizing
            
        Returns:
            The best score for the current position
        """
        self.nodes_visited += 1
        
        # Check for terminal states
        winner = self.check_winner(board)
        if winner is not None:
            if winner == -1:  # Computer wins
                return 1000
            elif winner == 1:  # Human wins
                return -1000
            else:  # Draw
                return 0
        
        # If maximum depth reached, evaluate board
        if depth == 0:
            return self.evaluate_board(board)
        
        player = -1 if is_maximizer else 1
        moves = self.get_possible_moves(board, player)
        
        if is_maximizer:
            best_score = float('-inf')
            for move in moves:
                new_board = self.apply_move(board, move, player)  # apply_move already does copying
                if new_board is None:
                    continue
                if self.prune: 
                    score = self.minimax_with_pruning(new_board, depth - 1,alpha,beta, False)
                else:
                    score = self.minimax_no_pruning(new_board, depth - 1, False)
                    
                best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for move in moves:
                new_board = self.apply_move(board, move, player)  # apply_move already does copying
                if new_board is None:
                    continue
                if self.prune:
                    score = self.minimax_with_pruning(new_board, depth -1,alpha,beta,True)
                else:
                    score = self.minimax_no_pruning(new_board, depth -1, True)
                best_score = min(score, best_score)
            return best_score
    
    def analyze_branching_factor(self, num_positions=20):
        """
        Analyze the average branching factor at different stages of the game.
        
        Args:
            num_positions: Number of positions to analyze
            
        Returns:
            DataFrame with branching factor statistics
        """
        results = []
        
        # Simulate games and collect branching factor data
        for _ in range(num_positions):
            game = PentagoGame(ui=False, print=False)
            move_number = 0
            game_history = []
            
            # Play a full game while recording branching factors
            while True:
                move_number += 1
                player = game.current_player
                
                # Get all possible moves
                moves = game.get_possible_moves(game.board, player)
                branching_factor = len(moves)
                
                # Record data
                empty_spaces = np.sum(game.board == 0)
                game_stage = "Early" if empty_spaces > 25 else "Middle" if empty_spaces > 10 else "Late"
                
                game_history.append({
                    'move_number': move_number,
                    'player': "Computer" if player == -1 else "Human",
                    'branching_factor': branching_factor,
                    'empty_spaces': empty_spaces,
                    'game_stage': game_stage
                })
                
                # Make a move
                if not moves:
                    break
                    
                if player == -1:
                    move = game.get_computer_move()
                else:
                    move = random.choice(moves)
                    
                game.board = game.apply_move(game.board, move, player)
                
                # Check for game end
                winner = game.check_winner(game.board)
                if winner is not None:
                    break
                
                game.current_player *= -1
            
            results.extend(game_history)
        
        return pd.DataFrame(results)

    def get_computer_move(self):
        start_time = time.time()
        best_move = None
        best_value = -inf
        
        moves = self.get_possible_moves(self.board, -1)
        if not moves:
            return None
        
        best_move = moves[0]  # Default move in case evaluation fails

        self.nodes_visited = 0  # Reset node counter before search
        
        for move in moves:
            if self.game_over:
                break
                
            new_board = self.apply_move(self.board, move, -1)
            if new_board is None:
                continue

            try:
                if self.prune:
                    # Use alpha-beta pruning with minimax
                    value = self.minimax_with_pruning(new_board, self.depth - 1, -inf, inf, False)
                else:
                    # without pruning
                    value = self.minimax_no_pruning(new_board, self.depth - 1, False)
                
                if value > best_value:
                    best_value = value
                    best_move = move
            except Exception as e:
                print(f"Error during minimax: {e}")
                # Continue with next move rather than setting value to -inf

        if self.print:
            print(f"Move took {time.time()-start_time:.2f}s, nodes visited: {self.nodes_visited}")
        
        return best_move

    def draw_text(self, text, center_pos, max_width):
        font_size = 24
        font = pygame.font.Font(None, font_size)
        text_surface = font.render(text, True, (0, 0, 0))

        text_width = text_surface.get_width()
        if text_width > max_width:
            scale_factor = max_width / text_width
            new_font_size = int(font_size * scale_factor)
            font = pygame.font.Font(None, new_font_size)
            text_surface = font.render(text, True, (0, 0, 0))

        text_rect = text_surface.get_rect(center=center_pos)
        self.screen.blit(text_surface, text_rect)

    def draw_board(self):
        self.screen.fill((0, 0, 0))

        for i in range(6):
            for j in range(6):
                x0 = j * 100
                y0 = i * 100

                if self.board[i][j] == 1:
                    pygame.draw.circle(self.screen, (255, 0, 0), (x0 + 50, y0 + 50), 40)
                elif self.board[i][j] == -1:
                    pygame.draw.circle(self.screen, (0, 0, 255), (x0 + 50, y0 + 50), 40)

                pygame.draw.rect(self.screen, (255, 255, 255), (x0, y0, 100, 100), 1)

        for i in [3, 6]:
            pygame.draw.line(self.screen, (255, 255, 255), (0, i * 100), (600, i * 100), 3)  # Horizontal
            pygame.draw.line(self.screen, (255, 255, 255), (i * 100, 0), (i * 100, 600), 3)  # Vertical

        # Show rotation buttons if in move_stage 2
        if self.move_stage == 2:
            self.highlight_selected_block()
            self.show_rotation_buttons()

        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])  # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.draw_text(
                "CLOCKWISE",
                self.buttons["rotate_cw"].center,
                self.buttons["rotate_cw"].width,
            )
            self.draw_text(
                "COUNTER-CLOCKWISE",
                self.buttons["rotate_ccw"].center,
                self.buttons["rotate_ccw"].width,
            )

    def click_handler(self, event):
        if self.game_over or self.current_player != 1:
            return

        x, y = event.pos
        if self.move_stage == 0:  # Place piece
            if x > 600:
                return  # clicks on control area
            col = x // 100
            row = y // 100
            if 0 <= row < 6 and 0 <= col < 6 and self.board[row][col] == 0:
                self.temp_piece = (row, col)
                self.board[row][col] = 1
                self.move_stage = 1
                self.draw_board()

        elif self.move_stage == 1:  # Select block
            if x > 600:
                return
            # which block was clicked
            block_x = 0 if x < 300 else 1
            block_y = 0 if y < 300 else 1
            self.selected_block = block_y * 2 + block_x
            self.move_stage = 2
            self.show_rotation_buttons()
            self.highlight_selected_block()

        elif self.move_stage == 2:  # Rotate
            if self.buttons["rotate_cw"].collidepoint(event.pos):
                self.apply_rotation("cw")
            if self.buttons["rotate_ccw"].collidepoint(event.pos):
                self.apply_rotation("ccw")

    def apply_rotation(self, direction):
        self.rotate_block(self.board, self.selected_block, direction)
        self.current_player = -1
        self.move_stage = 0
        self.selected_block = None
        self.temp_piece = None
        self.hide_rotation_buttons()
        self.draw_board()
        pygame.display.flip()
        self.check_game_over()
        pygame.time.delay(1000)
        self.play_computer_move()

    def highlight_selected_block(self):
        colors = [
            (255, 153, 153),
            (153, 255, 153),
            (153, 153, 255),
            (255, 255, 153),
        ]  # RGB colors

        row_start = (self.selected_block // 2) * 3
        col_start = (self.selected_block % 2) * 3

        pygame.draw.rect(
            self.screen,
            colors[self.selected_block],
            (col_start * 100, row_start * 100, 300, 300),
            5,
        )

    def play_computer_move(self):
        move = self.get_computer_move()
        if move and not self.game_over:
            new_board = self.apply_move(self.board, move, -1)
            if new_board is not None:
                self.board = new_board
                self.current_player = 1
                self.draw_board()
                pygame.display.flip()
                self.check_game_over()
            else:
                print("Invalid computer move!")

    def check_game_over(self):
        winner = self.check_winner(self.board)
        if winner is not None:
            self.game_over = True
            self.result = winner
            print("Game over! Result:", winner)
            if self.ui:
                self.show_game_over_message()

    def show_game_over_message(self):
        self.screen.fill((200, 200, 200))
        pygame.draw.rect(self.screen, (255, 255, 255), (100, 200, 500, 200))
        pygame.draw.rect(self.screen, (0, 0, 0), (100, 200, 500, 200), 3)

        result_text = f"Player {self.result} wins!" if self.result != 0 else "Draw!"
        text_surface = self.font_large.render(result_text, True, (255, 0, 0))
        self.screen.blit(text_surface, (250, 250))

        exit_text = self.font_small.render("Click anywhere to exit", True, (0, 0, 0))
        self.screen.blit(exit_text, (230, 350))
        pygame.display.flip()

    def play(self):
        if self.ui:
            running = True
            while running:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        self.click_handler(event)
                self.draw_board()
                pygame.display.flip()
            pygame.quit()
            return self.result
        else:
            while not self.game_over:
                if self.print:
                    self.print_board()
                winner = self.check_winner(self.board)
                if winner is not None:
                    self.result = winner
                    self.game_over = True
                    return winner

                if self.current_player == 1:
                    move = random.choice(self.get_possible_moves(self.board, 1))
                else:
                    move = self.get_computer_move()

                self.board = self.apply_move(self.board, move, self.current_player)
                self.current_player *= -1
            return self.result

    def print_board(self):
        if self.print == False:
            return
        print("-" * 25)
        for row in self.board:
            print(" ".join(f"{x:2}" for x in row))
        print("-" * 25)

Run 20 Games without Pruning and see the results:(the result was 20 win for computer and 4:30 time!!!!!!!!)

In [80]:
# game = PentagoGame(ui=False, print=False, depth=2)  # depth=2 for faster
# numGames = 20
# numWins, numTies, numLosses = 0, 0, 0
# for i in range(numGames):
#     result = game.play()
#     if result == -1:
#         numWins += 1
#     elif result == 0:
#         numTies += 1
#     else:
#         numLosses += 1

# print(f"{numWins} wins, {numTies} ties, {numLosses} losses")

We change the depth to One:

In [81]:
game = PentagoGame(ui=False, print=False, depth=1)  # depth=2 for faster
numGames = 20
numWins, numTies, numLosses = 0, 0, 0
for i in range(numGames):
    result = game.play()
    if result == -1:
        numWins += 1
    elif result == 0:
        numTies += 1
    else:
        numLosses += 1

print(f"{numWins} wins, {numTies} ties, {numLosses} losses")

20 wins, 0 ties, 0 losses


Now we use the pruning to boost the speed of our algorithm:

In [92]:
game = PentagoGame(ui=False, print=False, depth=2, prune=True)  # depth=2 for faster
numGames = 20
numWins, numTies, numLosses = 0, 0, 0
for i in range(numGames):
    result = game.play()
    if result == -1:
        numWins += 1
    elif result == 0:
        numTies += 1
    else:
        numLosses += 1

print(f"{numWins} wins, {numTies} ties, {numLosses} losses")

20 wins, 0 ties, 0 losses


Finding the avg time and visited nodes:

In [ ]:
depths_to_test = [1, 2, 3, 4]
results = []

for depth in depths_to_test:
    game = PentagoGame(ui=False, print=False, depth=depth, prune=True)
    num_wins = 0
    total_nodes = 0
    start_time = time.time()
    
    for _ in range(20):  
        result = game.play()
        if result == -1:  
            num_wins += 1
        total_nodes += game.nodes_visited
    
    avg_time = (time.time() - start_time) / 20
    avg_nodes = total_nodes / 20
    results.append((depth, num_wins/20, avg_time, avg_nodes))


print("Depth | Win Rate | Avg Time | Avg Nodes")
for r in results:
    print(f"{r[0]:5} | {r[1]:7.2%} | {r[2]:7.3f}s | {r[3]:,.0f}")

In [98]:
def get_ordered_moves(self, board, player):
    moves = []
    for i in range(6):
        for j in range(6):
            if board[i][j] == 0:
                for block in range(4):
                    for dir in ["cw", "ccw"]:
                        moves.append((i, j, block, dir))
    
    moves.sort(key=lambda m: self.move_heuristic(board, m, player), reverse=True)
    return moves

def move_heuristic(self, board, move, player):
    row, col, block, dir = move
    score = 0
    
    new_board = self.apply_move(self.copy_board(board), move, player)
    if self.check_winner(new_board) == player:
        return float('inf')
    
    if (row, col) in [(2,2), (2,3), (3,2), (3,3)]:
        score += 3
        
    score += self.count_potential_lines(new_board, player) * 2
    
    return score

In [99]:
def expectimax(board, depth, is_maximizer):
    if depth == 0 or game_over(board):
        return evaluate(board)
    
    if is_maximizer:
        value = -infinity
        for move in get_moves(board, AI_PLAYER):
            new_board = make_move(board, move)
            value = max(value, expectimax(new_board, depth-1, False))
        return value
    else:
        value = 0
        moves = get_moves(board, OPPONENT)
        for move in moves:
            new_board = make_move(board, move)
            value += expectimax(new_board, depth-1, True) * (1/len(moves))
        return value